# Calculations of charged defects in 2D materials
a tutorial by Anne Marie Tan

Some things to note before we get started:
* Download the python scripts from the [github repository](https://github.com/aztan2/charged-defects-framework) and place them in your home directory on hipergator. Export this location to your PYTHONPATH in `~/.bashrc`.
* You will need to launch this notebook from a virtual environment on hipergator in which you have installed python packages like numpy, matplotlib, pymatgen, pandas, nglview (if you want to use the built-in crystal viewer), and of course jupyterlab.
* Follow the instructions in the [document on the Hennig group google drive](https://drive.google.com/file/d/15qzXZkK6Wrmor-9JOuGI_-nMmcHZsAoe/view?usp=sharing) to start a Jupyter notebook within a SLURM job on hipergator and connect to it from the web browser running on your local computer.
* For the purpose of this tutorial, I will try to keep everything self-contained by executing all commands within the notebook, including navigating directories, executing python funtions and scripts, etc. However, when you actually apply this to a new system, you will probably find it easier to do some of these directly from the command line.
* Please go through the tutorial **"Calculating structure and properties of pristine 2D materials"** before starting on this tutorial.

The main quantity that we’re interested in calculating is the defect formation energy $E^f[X^q]$, which we can compute in DFT using the supercell approach and the following equation:

<div>
<img src="tutorial_images/eqn_Eform.png" width="400"/>
</div>

where $E_{\textrm{tot}}[X^q]$ and $E_{\textrm{tot}}[\textrm{pristine}]$ are the total DFT-derived energies of the supercell containing the defect $X$ and the pristine supercell respectively, $n_i$ is the number of atoms of species $i$ added ($n_i$ > 0) or removed ($n_i$ < 0) by the defect, $\mu_i$ is the corresponding chemical potential of the species, and $E_{\textrm{F}}$ is the Fermi energy. 
The final term $E_{\textrm{corr}}$ contains corrections to the formation energy due to electrostatic interactions with periodic images and the implicit compensating background charges which are introduced in supercell calculations using plane-wave DFT approaches. We will evaluate this term using the method developed by [Freysoldt and Neugebauer](https://doi.org/10.1103/PhysRevB.97.205425).

(This tutorial is meant to focus on how to perform the calculations and corrections, hence I will no go into detail about the theory here. You are recommended to read up on your own about (1) the origin of the artefacts that we need to correct for, and (2) how this correction scheme works.)

Based on the equation above, the DFT calculations that are required to compute the formation energy are:
* $E_{\textrm{tot}}[\textrm{pristine}]$
* $E_{\textrm{tot}}[X^q]$ for every defect of interest $X$ in different charge states $q$
* the relevant $\mu_i$

So let's get started!

In [2]:
import pandas

from qdef2d.io.database import parse_energies

from qdef2d.defects import setup_defect_calcs, calc_Eform_uncorr, calc_Eform_corr
from qdef2d.defects.corrections import apply_corrections_2d, parse_corrections

In [17]:
# any time one of the imported files is modified, it must be reloaded
import importlib
importlib.reload(calc_Eform_corr)
#importlib.reload(apply_corrections_2d)
importlib.reload(parse_corrections)

<module 'qdef2d.defects.corrections.parse_corrections' from '/home/annemarietan/myscripts/qdef2d/defects/corrections/parse_corrections.py'>

### 0.      Determine equilibrium lattice constants:

For the defect calculations, we will fix the lattice parameters and only relax the atom positions. This ensures that we have a consistent reference for all our calculations (with/without defects) and models the system in which the defect concentration is in the dilute limit.

* Based on the pristine unit cell calculations you did in the **"Calculating structure and properties of pristine 2D materials"** tutorial, estimate the converged values of the in-plane lattice parameter(s). It should be around 3.18 Å for monolayer MoS$_2$ evaluated with PBE.

* Prepare a set of unit cell POSCARS with these lattice constants at different vacuum spacings (e.g. 10, 15, 20 Å). You can do this by just slightly modifying the lattice vectors at the top of your existing corresponding unit cell POSCARs. Name each of the new POSCARs `POSCAR_vac_<vacuum spacing>` (this naming convention is important!).

### 1.      Total energy of a pristine supercell:

This calculation is pretty straightforward. In fact, haven’t we already computed the energy of the pristine monolayer in the previous tutorial? Why do we need to do it again? 

While it is true that we could simply compute the total energy of a pristine supercell by multiplying the energy of a pristine unit cell accordingly, our later calculations will also require the potential (LOCPOT) of the pristine supercell, so we still need to perform the pristine supercell calculation anyway.

* Create a new directory for this set of calculations. Let's assume I've named it `pristine`. \
Create a subdirectory within this for calculations with GGA functional. 

In [8]:
%cd /blue/hennig/annemarietan/test/MoS2
%mkdir pristine/GGA

/blue/hennig/annemarietan/test/MoS2


* Copy the POSCARs you made in Step 0 into this directory

In [9]:
%cp unitcell/POSCAR_vac_* pristine/GGA/.
%cd pristine/GGA
%ls

/blue/hennig/annemarietan/test/MoS2/pristine/GGA
POSCAR_vac_10  POSCAR_vac_15  POSCAR_vac_20


* Prepare the POTCAR by concatenating the appropriate element POTCARs. \
In the following example, we will be creating a S vacancy in MoS$_2$, therefore we will only require the `Mo_pv` POTCAR for Mo and `S` POTCAR for S, as before. \
If you are introducing a dopant or impurity of a different species, you will have to concatenate the appropriate POTCAR. Remember, pymatgen orders the elements in the POSCAR by increasing electronegativity, hence the element POTCARs must be concatenated in the same order!

In [10]:
%cat /home/annemarietan/POTCAR/POT_GGA_PAW_PBE/Mo_pv/POTCAR /home/annemarietan/POTCAR/POT_GGA_PAW_PBE/S/POTCAR > POTCAR
%ls

POSCAR_vac_10  POSCAR_vac_15  POSCAR_vac_20  POTCAR


* Run the following python script to generate all the input files for our pristine supercell calculations: \
(run `help(setup_defect_calcs.setup)` or `python setup_defect_calcs.py --h` to see the description of the required and optional arguments for this script)

In [18]:
setup_defect_calcs.setup("/blue/hennig/annemarietan/test/MoS2/pristine/GGA/", 
                    qs=[0], cells=[[3,3,1],[4,4,1]], vacs=[10,15,20], 
                    kppa=440,bulkref=True)
# python setup_defect_calcs.py /path/to/pristine/GGA --qs 0 --cells 3x3x1 4x4x1 --vacs 10 15 20 --kppa 440 --bulkref

INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/pristine/GGA/charge_0/3x3x1/vac_10
INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/pristine/GGA/charge_0/3x3x1/vac_15


ref_3x3x1_10


INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/pristine/GGA/charge_0/3x3x1/vac_20


ref_3x3x1_15


INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/pristine/GGA/charge_0/4x4x1/vac_10


ref_3x3x1_20


INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/pristine/GGA/charge_0/4x4x1/vac_15


ref_4x4x1_10


INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/pristine/GGA/charge_0/4x4x1/vac_20


ref_4x4x1_15
ref_4x4x1_20


* This should have created 6 sub-directories in the `pristineref/GGA/` main directory, following the directory structure of `charge/supercell_size/vacuum_spacing`. This is the default directory structure that all my scripts have been designed to work with. Check that each sub-directory contains a POSCAR, POTCAR, INCAR, KPOINTS, and submission script.

In [19]:
%cd charge_0/3x3x1/vac_10
%ls

/blue/hennig/annemarietan/test/MoS2/pristine/GGA/charge_0/3x3x1/vac_10
INCAR  KPOINTS  POSCAR  POTCAR  submitVASP.sh


In [20]:
%cat POSCAR

Mo9 S18
1.0
9.541941 0.000000 0.000000
-4.770970 8.263562 0.000000
0.000000 0.000000 13.110000
Mo S
9 18
direct
0.222222 0.111111 0.500000 Mo
0.222222 0.444444 0.500000 Mo
0.222222 0.777778 0.500000 Mo
0.555556 0.111111 0.500000 Mo
0.555556 0.444444 0.500000 Mo
0.555556 0.777778 0.500000 Mo
0.888889 0.111111 0.500000 Mo
0.888889 0.444444 0.500000 Mo
0.888889 0.777778 0.500000 Mo
0.111111 0.222222 0.619112 S
0.111111 0.555556 0.619112 S
0.111111 0.888889 0.619112 S
0.444444 0.222222 0.619112 S
0.444444 0.555556 0.619112 S
0.444444 0.888889 0.619112 S
0.777778 0.222222 0.619112 S
0.777778 0.555556 0.619112 S
0.777778 0.888889 0.619112 S
0.111111 0.222222 0.380888 S
0.111111 0.555556 0.380888 S
0.111111 0.888889 0.380888 S
0.444444 0.222222 0.380888 S
0.444444 0.555556 0.380888 S
0.444444 0.888889 0.380888 S
0.777778 0.222222 0.380888 S
0.777778 0.555556 0.380888 S
0.777778 0.888889 0.380888 S


In [22]:
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.io.vasp import Poscar

structure = Poscar.from_file('POSCAR').structure
atoms = AseAtomsAdaptor.get_atoms(structure)
ngl_handler = view(atoms, viewer='ngl')
ngl_handler.view.add_representation('ball+stick', selection='all')
ngl_handler.view.center()
ngl_handler

_ColormakerRegistry()

* Submit your jobs!\
Note that the default in the `gen_submit.py` script is to submit to the priority queue (`hennig`). The number of cores available on the priority queue is limited and shared among everyone in the group, so you don't want to hog too many cores all at once. You may change the default in the python script or edit the submission script directly to submit to the burst queue (`hennig-b`) instead.

* When your jobs are done, what’s an easy sanity check you can do? (hint: see the discussion at the beginning of Step 1)

### 2.      Total energy of a defected supercell:

Finally, we get to the real meat of our calculations! 

Let’s first consider a simple intrinsic defect, a S vacancy in MoS$_2$.

* Create a main defect directory for this set of calculations. Let’s assume I’ve named it `Svac`. \
Again, create a subdirectory within this for calculations with GGA functional.

In [23]:
%cd /blue/hennig/annemarietan/test/MoS2
%mkdir Svac/GGA

/blue/hennig/annemarietan/test/MoS2
mkdir: cannot create directory ‘Svac/GGA’: File exists


* Copy the same POTCAR and unit cell POSCARs as used in step 1 into this directory.

In [24]:
%cp pristine/GGA/PO* Svac/GGA/.
%cd Svac/GGA
%ls

/blue/hennig/annemarietan/test/MoS2/Svac/GGA
POSCAR_vac_10  POSCAR_vac_15  POSCAR_vac_20  POTCAR


* We will specify the number, type, and position of the defect(s) to create in an `initdefect.json` file. \
For a simple vacancy, this is very straightforward. For example, to create a S vacancy by removing the S atom at the last position in the POSCAR, simply create the file with the following lines: 

```
{
	"def1": {"type": "vac", "index": -1, "species": "S"}
}
```

This will tell the script to remove the S atom at index -1 (in python, negative indices will count backwards from the end of the list). To learn how to specify other types of defects, please refer to [the additional document that I have to write].

You can simply copy the lines above into your text editor of choice on hipergator and save the file as `initdefect.json` (the name is important!). However, in an attempt to keep this notebook as self-contained as possible, I also provide a small python script below which will do the same thing.

In [25]:
import json

defects = {"def1":{"type": "vac", "index": -1, "species": "S"}}

with open("initdefect.json", 'w') as file:
    file.write(json.dumps(defects,indent=4))

In [30]:
%cat initdefect.json

cat: initdefect.json: No such file or directory


* Now, you have all you need to run the following python script to generate all the input files for our defect calculations:

In [31]:
setup_defect_calcs.setup("/blue/hennig/annemarietan/test/MoS2/Svac/GGA/", 
                          qs=[0,-1,1], cells=[[3,3,1],[4,4,1]], vacs=[10,15,20], 
                          kppa=440)
# python setup_defect_calcs.py /path/to/Svac/GGA --qs 0 -1 1 --cells 3x3x1 4x4x1 --vacs 10 15 20 --kppa 440

INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/Svac/GGA/charge_0/3x3x1/vac_10
INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/Svac/GGA/charge_0/3x3x1/vac_15


vac_S_0_3x3x1_10


INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/Svac/GGA/charge_0/3x3x1/vac_20


vac_S_0_3x3x1_15


INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/Svac/GGA/charge_0/4x4x1/vac_10


...

vac_S_1_3x3x1_20


INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/Svac/GGA/charge_1/4x4x1/vac_15


vac_S_1_4x4x1_10


INFO:generating input files in /blue/hennig/annemarietan/test/MoS2/Svac/GGA/charge_1/4x4x1/vac_20


vac_S_1_4x4x1_15
vac_S_1_4x4x1_20


* This should have created 3 x 2 x 3 = 18 sub-directories in the `Svac/GGA/` main directory, following the directory structure of `charge/supercell_size/vacuum_spacing`. Check that each sub-directory contains a POSCAR, POTCAR, INCAR, KPOINTS, `defectproperty.json` and submission script.

In [32]:
%cd charge_-1/3x3x1/vac_10
%ls

/blue/hennig/annemarietan/test/MoS2/Svac/GGA/charge_-1/3x3x1/vac_10
defectproperty.json  INCAR  KPOINTS  POSCAR  POTCAR  submitVASP.sh


* You should see that the POSCARs now have 1 fewer S atom. You can also visualize the structure in VESTA/CrystalMaker/etc. to check that you did indeed create the defect as intended. This is particularly important when you create more complex defects (e.g. interstitials, defect clusters, etc.) for which the above method of defining the defect site is more complicated.

In [33]:
#from ase.visualize import view
#from pymatgen.io.ase import AseAtomsAdaptor
#from pymatgen.io.vasp import Poscar

structure = Poscar.from_file('POSCAR').structure
atoms = AseAtomsAdaptor.get_atoms(structure)
ngl_handler = view(atoms, viewer='ngl')
ngl_handler.view.add_representation('ball+stick', selection='all')
ngl_handler.view.center()
ngl_handler

* Charged systems are specified in VASP by setting the `NELECT` (number of electrons) tag in the INCAR. By default, VASP sets `NELECT` to be $\sum_i n_i Z_i$, where $n_i$ and $Z_i$ are the numbers of atoms and valence electrons of element $i$, respectively. \
For example, a neutral system with 9 Mo and 17 S atoms should have 9 x 12 + 17 x 6 = 210 valence electrons. We would specify a negatively charged system by adding electrons; for example, by setting `NELECT = 211` in the INCAR. You can verify that this has been taken care of by the script by comparing the INCARs for the defect in different charge states.

In [34]:
%cat INCAR

PREC = Accurate
ALGO = Fast
LREAL = Auto
ISYM = 0
NELECT = 211
ENCUT = 520
NELM = 120
EDIFF = 1e-05
ISIF = 2
IBRION = 2
NSW = 100
ISMEAR = 1
SIGMA = 0.1
ISPIN = 2
MAGMOM = 9*5.0 17*0.6
LPLANE = True
NCORE = 8
LWAVE = False
LCHARG = False
LMAXMIX = 4
LORBIT = 11
LVHAR = True


* The `defectproperty.json` is a new file we have not encountered before. It contains some basic information about the defect and the supercell it has been created in, organized in a standarized dictionary form that is easily accessible by other python scripts that will later parse through the directories extracting relevant information and applying the correction scheme. Let's take a look at it:

In [35]:
%cat defectproperty.json

{
    "charge": -1,
    "defect_type": [
        "vac_S"
    ],
    "defect_site": [
        [
            0.7777777777777777,
            0.8888888888888888,
            0.38088767112
        ]
    ],
    "lattice": {
        "@module": "pymatgen.core.lattice",
        "@class": "Lattice",
        "matrix": [
            [
                9.541940940734781,
                0.0,
                0.0
            ],
            [
                -4.7709704703673905,
                8.263561989784584,
                0.0
            ],
            [
                0.0,
                0.0,
                13.11
            ]
        ]
    },
    "supercell": [
        3,
        3,
        1
    ],
    "vacuum": 10
}

* If everything looks good, submit your jobs!\
Note, as before, that you can change the defaults for the queue, number of nodes, memory, and runtime requested for each job. 

### 3.      Chemical potentials:

The defect formation energy depends on the choice of chemical potential reference(s). For the example of a S vacancy in MoS$_2$, the relevant chemical potential is that of a S atom. However, the chemical potential of a S atom depends on the environment that it is in.

In MoS$_2$, the chemical potentials of Mo ($\mu_{\textrm{Mo}}$) and S ($\mu_{\textrm{S}}$) are related by the stability of the MoS$_2$ phase, i.e. $\mu_{\textrm{Mo}} + 2\mu_{\textrm{S}} = \mu_{\textrm{MoS$_2$}}$. Within this constraint, the chemical potentials can be varied, subject to bounds determined by the competing phases.

* Let's check the phase diagram for the Mo-S system to see what are the relevant competing phases. The following phase diagram was obtained from https://materialsproject.org.
<div>
<img src="tutorial_images/phase_diagram.png" width="800"/>
</div>

Turns out, this system is pretty straightforward, with the only competing phases being the pure elements. Hence, we can define the chemical potentials in two limits:

* Mo-rich (S-poor) limit: $\mu_{\textrm{Mo}} = \mu_{\textrm{bcc Mo}}$; $\mu_{\textrm{S}} = (\mu_{\textrm{MoS$_2$}} - \mu_{\textrm{bcc Mo}})/2$
* S-rich (Mo-poor) limit: $\mu_{\textrm{Mo}} = \mu_{\textrm{MoS$_2$}} - 2\mu_{\textrm{elem S}}$; $\mu_{\textrm{S}} = \mu_{\textrm{elem S}}$

There may be some ambiguity in choosing the reference phase for the elemental S, so for simplicity, we'll stick with the Mo-rich limit for now. $\mu_{\textrm{MoS$_2$}}$ is simply the energy of a formula unit of monolayer MoS$_2$, which we already have. Therefore, all we need is the energy per Mo atom in bcc Mo.

* Download the input files from Materials Project (mp-129) and run this calculation. If the unit cell has more than 1 atom, don't forget to divide the total energy accordingly!

### 4.      (Uncorrected) defect formation energy:

We are done with the DFT calculations at this point, and can now start pulling together the relevant computed quantities to evaluate the formation energy of a S vacancy defect in monolayer MoS$_2$!

Remember, the equation for the defect formation energy is:

<div>
<img src="tutorial_images/eqn_Eform.png" width="400"/>
</div>

First, let’s see what happens when we don’t apply the correction, i.e. evaluate only the first 4 terms in the equation:

For the purpose of this tutorial, I have run some calculations beforehand and copied some of my output files into the corresponding directories `/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/` and `/orange/hennig/annemarietan/precalculated_for_demo/MoS2/pristine/GGA/` so that I can demonstrate the post-processing steps without waiting for all your jobs to complete. These calculations were performed with lattice constant $a_0$ = 3.180647 Å, which should be close to the equilibrium lattice constant you arrived at above. Regardless, the most important thing is consistency, so as long as you stick with the same choice for all your calculations, the final results should not be significantly different.

* Run the following python script to parse the total energies from the completed DFT calculations and save them in a pandas dataframe:

In [42]:
%cd /orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA

parse_energies.parse("/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/",
                    "/orange/hennig/annemarietan/precalculated_for_demo/MoS2/pristine/GGA/",
                    "Eform_Svac.xlsx")
# python parse_energies.py /path/to/Svac/GGA /path/to/pristine/GGA Eform_Svac.xlsx

INFO:parsing neutral 3x3x1 vac_15


/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA


INFO:parsing neutral 3x3x1 vac_20
INFO:parsing neutral 4x4x1 vac_20
INFO:parsing charge_-1 3x3x1 vac_15
INFO:parsing charge_-1 3x3x1 vac_20
INFO:parsing charge_-1 4x4x1 vac_20
INFO:parsing charge_1 3x3x1 vac_15
INFO:parsing charge_1 3x3x1 vac_20
INFO:parsing charge_1 4x4x1 vac_20
DEBUG:Total time taken (s): 10.42


* The pandas dataframe should be written out to an excel file in this directory. We can take a look at its contents: \
The values in columns `E_def` and `E_bulk` are respectively $E_{\textrm{tot}}[X^q]$ and $E_{\textrm{tot}}[\textrm{pristine}]$ in the above equation.

In [43]:
df = pandas.read_excel("Eform_Svac.xlsx", sheet_name=None)
for q in df.keys():
    print (q)
    display(df[q])

charge_0


,vacuum,supercell,N,1/N,E_def,E_bulk
0,vac_15,3x3x1,27,0.037037,-189.728601,-196.531426
1,vac_20,3x3x1,27,0.037037,-189.725299,-196.528449
2,vac_20,4x4x1,48,0.020833,-342.600831,-349.384544


charge_-1


,vacuum,supercell,N,1/N,E_def,E_bulk
0,vac_15,3x3x1,27,0.037037,-189.757534,-196.531426
1,vac_20,3x3x1,27,0.037037,-190.233532,-196.528449
2,vac_20,4x4x1,48,0.020833,-343.515495,-349.384544


charge_1


,vacuum,supercell,N,1/N,E_def,E_bulk
0,vac_15,3x3x1,27,0.037037,-187.400635,-196.531426
1,vac_20,3x3x1,27,0.037037,-186.104404,-196.528449
2,vac_20,4x4x1,48,0.020833,-339.461205,-349.384544


* Next, we will need the relevant chemical potential(s) and VBM energies (we will evaluate the defect formation energies first at $E_{\textrm{F}} = E_{\textrm{VBM}}$ before taking into account the Fermi energy dependence). \
These quantities can be manually extracted from the unit cell calculations you have performed in previous steps, plugged into the equation, and the uncorrected defect formation energy evaluated in this way.

* However, I will demonstrate how this can be done in a more automated manner IF we have previously calculated and stored these values in a database (e.g. Materials Web, which we hope to do). Since such a database does not yet exist, I have made a "mock database" at `/orange/hennig/annemarietan/precalculated_for_demo/minidb` with entries of the following form:

In [44]:
%cd /orange/hennig/annemarietan/precalculated_for_demo/minidb

%cat MoS2.json # database entry for monolayer MoS2

/orange/hennig/annemarietan/precalculated_for_demo/minidb
{
    "GGA": {
        "mu": -21.83641285,
        "eps_ave": 17.18,
        "d_slab": 5.4,
        "Egap": 1.6787999999999998,
        "vac_10": {
            "VBM": -0.0395
        },
        "vac_15": {
            "VBM": -1.6657
        },
        "vac_20": {
            "VBM": -2.5881
        }
    }
}

In [45]:
%cat Mo.json # database entry for BCC Mo

{
    "GGA": {
        "mu": -10.9232799,
        "Egap": 0,
        "VBM": 12.3728
    }
}

In [46]:
%cat S.json # database entry for S

{
    "GGA": {
        "mu_Mo-rich (0.5*MoS2-0.5*Mo)": -5.456566475
    }
}

Note that for S, the chemical potential value has been derived from the equation for the Mo-rich limit (see Section 3 above), and does not reflect the energy of a particular phase of elemental S. Hence, by plugging this value into the equation for the defect formation energy, we will be evaluating the formation energy specifically in the Mo-rich (S-poor) limit.

* The following python script uses the total energies stored in the above pandas dataframe and the chemical potential(s) and VBMs from the database to evaluate the first 4 terms in the equation for the defect formation energy, i.e. the defect formation energy before adding the correction term. 

In [60]:
%cd /orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA

calc_Eform_uncorr.calc("MoS2",
                       "/orange/hennig/annemarietan/precalculated_for_demo/minidb",
                       "/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/",
                        "Eform_Svac.xlsx",
                        "Mo-rich")
# python calc_Eform_uncorr.py MoS2 /path/to/database /path/to/Svac/GGA Eform_Svac.xlsx Mo-rich

INFO:Atoms added/removed: -1*S
INFO:Using chemical potential mu_Mo-rich (0.5*MoS2-0.5*Mo) from S.json
INFO:Using chemical potential mu_Mo-rich (0.5*MoS2-0.5*Mo) from S.json
INFO:Using chemical potential mu_Mo-rich (0.5*MoS2-0.5*Mo) from S.json


/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA


In [62]:
df = pandas.read_excel("Eform_Svac.xlsx", sheet_name=None)
for q in df.keys():
    print (q)
    display(df[q])

charge_0


,vacuum,supercell,N,1/N,E_def,E_bulk,mu_S_Mo-rich,VBM,E_form_corr
0,vac_15,3x3x1,27,0.037037,-189.728601,-196.531426,-5.456566,-1.6657,1.346259
1,vac_20,3x3x1,27,0.037037,-189.725299,-196.528449,-5.456566,-2.5881,1.346584
2,vac_20,4x4x1,48,0.020833,-342.600831,-349.384544,-5.456566,-2.5881,1.327147


charge_-1


,vacuum,supercell,N,1/N,E_def,E_bulk,mu_S_Mo-rich,VBM,E_form_uncorr
0,vac_15,3x3x1,27,0.037037,-189.757534,-196.531426,-5.456566,-1.6657,2.983025
1,vac_20,3x3x1,27,0.037037,-190.233532,-196.528449,-5.456566,-2.5881,3.426451
2,vac_20,4x4x1,48,0.020833,-343.515495,-349.384544,-5.456566,-2.5881,3.000583


charge_1


,vacuum,supercell,N,1/N,E_def,E_bulk,mu_S_Mo-rich,VBM,E_form_uncorr
0,vac_15,3x3x1,27,0.037037,-187.400635,-196.531426,-5.456566,-1.6657,2.008525
1,vac_20,3x3x1,27,0.037037,-186.104404,-196.528449,-5.456566,-2.5881,2.379379
2,vac_20,4x4x1,48,0.020833,-339.461205,-349.384544,-5.456566,-2.5881,1.878673


You should notice that the formation energy for the neutral defect is pretty well-converged (e.g. within 10s of meV) across all supercell sizes and vacuum spacings. A slight in-plane supercell size dependence is to be expected due to the (mostly elastic) interactions between the defect and its periodic images, which decreases with increasing in-plane supercell size. The vacuum dependence should be negligible in the case of the neutral defect.

However, you should see that this is NOT the case for either of the charged defects! The uncorrected formation energy for the charged defects diverges, especially with respect to vacuum spacing. This has to do with the uniform compensating background charge implicitly introduced by VASP during the calculation to neutralize the supercell, which gives rise to an unphysical quadratic potential in the vacuum region, leading to an unphysical linear divergence in the energy of charged systems.

*This is why we need to include the correction term!* So, we press on...

### 5.      Evaluating the charge correction:

We will utilize the [charge correction method developed by Freysoldt and Neugebauer](https://doi.org/10.1103/PhysRevB.97.205425) to determine the correction $E_{\textrm{corr}}$ which corrects for the artefacts introduced by treating charged defects in the periodic supercell approach. Put simply, this method constructs a surrogate model system in which the charge is approximated by a gaussian charge, and calculates the extra energy contributions due to the constraints of the periodic supercell approach acting on this surrogate model system as an estimate for the effect of these constraints on the true DFT system. Please refer to the paper linked above for more details.

* Download the sxdefectalign2d executable from https://sxrepo.mpie.de/projects/sphinx-add-ons/files and place it in your home directory on hipergator so that it is accessible at `~/sxdefectalign2d`. If you place it somewhere else, you will have to change the default path in the `get_alignment_correction_2d.py` script.

* Run the following python script to apply the correction scheme (this will take a few minutes so be patient):

In [11]:
%cd /orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA

apply_corrections_2d.apply_all("/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/",
                               "/orange/hennig/annemarietan/precalculated_for_demo/MoS2/pristine/GGA/",
                               dbentry="/orange/hennig/annemarietan/precalculated_for_demo/minidb/MoS2.json")
# python /path/to/apply_corrections_2d.py /path/to/Svac/GGA /path/to/pristineref/GGA --dbentry <>
# or
# python /path/to/apply_corrections_2d.py /path/to/Svac/GGA /path/to/pristineref/GGA --eps_slab <> --d_slab <>

INFO:Using slab properties from /orange/hennig/annemarietan/precalculated_for_demo/minidb/MoS2.json
INFO:eps_slab: 17.18 ; d_slab: 5.40
INFO:applying correction to calculations in /orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/charge_-1/3x3x1/vac_15/


/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA


INFO:shift charge in +z direction; try shift = 1.00000000
INFO:shift charge in +z direction; try shift = 2.00000000
INFO:shift charge in +z direction; try shift = 3.00000000
INFO:shift charge in +z direction; try shift = 4.00000000
INFO:shift charge in -z direction; try shift = 3.50000000
INFO:shift charge in -z direction; try shift = 3.25000000
INFO:shift charge in +z direction; try shift = 3.37500000
INFO:shift charge in -z direction; try shift = 3.31250000
INFO:DONE! shift = 3.31250000 & alignment correction = -0.11550000
INFO:applying correction to calculations in /orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/charge_-1/3x3x1/vac_20/
INFO:shift charge in +z direction; try shift = 1.00000000
INFO:shift charge in +z direction; try shift = 2.00000000
INFO:shift charge in +z direction; try shift = 3.00000000
INFO:shift charge in +z direction; try shift = 4.00000000
INFO:shift charge in -z direction; try shift = 3.50000000
INFO:shift charge in -z direction; try shift =

You will notice that there's a lot of "shifting" of a charge going on as the correction scheme is being applied. Essentially, what we're trying to do here is to determine the effective position of the model gaussian charge such that the long-range potential generated by the model system matches that in our DFT system as best as possible, so that the correction energy evaluated using the model system will be a good estimate for the correction energy of our actual system.

* Let's take a look at the outputs from applying the correction scheme:

In [12]:
%cd /orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/charge_-1/4x4x1/vac_20/correction
%ls

/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/charge_-1/4x4x1/vac_20/correction
alignment_0.png  alignment_4.png  alignment_8.png     system.sx
alignment_1.png  alignment_5.png  correction          vline-eV.dat
alignment_2.png  alignment_6.png  getalign.log
alignment_3.png  alignment_7.png  system-profile.dat


* You can check the `alignment*.png` plots to verify that the model potential has been well aligned with the DFT potential. For the -1 charged S vacancy, the final plot should look something like this:
 <div>
<img src="tutorial_images/alignment_8.png" width="500"/>
</div>

* The correction energy $E_\textrm{corr}$ can be found in the last line of the `correction` file:

In [13]:
%cat correction

VASP mesh: 192 x 192 x 360
cell defect = [a1={24.0422,0,0},a2={-12.0211,20.8212,0},a3={0,0,43.6716}]
VASP mesh: 192 x 192 x 360
cell bulk = [a1={24.0422,0,0},a2={-12.0211,20.8212,0},a3={0,0,43.6716}]
Cutoff = 38.2193 Ry
N = 176
Shifting charge by 3.0625
Shifting potential by -0.0511767 eV
--- Periodic
Q=1
--- Isolated
Isolated from 10.917915075 to 32.753745225 (88 points)
Q=1
No dielectric interface detected.
Dielectric position shift: -9.61056400233
zL = 10.917915075 eps = 1
E-field dependence left:  zeff = 17.0370427967
zR = 32.753745225 eps = 1
E-field dependence right: zeff = 26.647606799
---
isolated energy = 0.84637625122 eV
periodic energy = 0.889408779309 eV
iso - periodic energy = -0.0430325280888 eV


* The following python script parses the correction term from each of the `correction` files and inserts them accordingly into the same pandas dataframe/excel file as before:

In [18]:
%cd /orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA

parse_corrections.parse("/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/",
                        "Eform_Svac.xlsx")
# python /path/to/parse_corrections.py /path/to/Svac/GGA Eform_Svac.xlsx

INFO:parsing charge_-1 3x3x1 vac_15
INFO:parsing charge_-1 3x3x1 vac_20
INFO:parsing charge_-1 4x4x1 vac_20
INFO:parsing charge_1 3x3x1 vac_15
INFO:parsing charge_1 3x3x1 vac_20
INFO:parsing charge_1 4x4x1 vac_20


/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA


In [19]:
df = pandas.read_excel("Eform_Svac.xlsx", sheet_name=None)
for q in df.keys():
    print (q)
    display(df[q])

charge_0


,vacuum,supercell,N,1/N,E_def,E_bulk,mu_S_Mo-rich,VBM,E_form_corr
0,vac_15,3x3x1,27,0.037037,-189.728601,-196.531426,-5.456566,-1.6657,1.346259
1,vac_20,3x3x1,27,0.037037,-189.725299,-196.528449,-5.456566,-2.5881,1.346584
2,vac_20,4x4x1,48,0.020833,-342.600831,-349.384544,-5.456566,-2.5881,1.327147


charge_-1


,vacuum,supercell,N,1/N,E_def,E_bulk,mu_S_Mo-rich,VBM,E_form_uncorr,E_corr
0,vac_15,3x3x1,27,0.037037,-189.757534,-196.531426,-5.456566,-1.6657,2.983025,0.015273
1,vac_20,3x3x1,27,0.037037,-190.233532,-196.528449,-5.456566,-2.5881,3.426451,-0.413040
2,vac_20,4x4x1,48,0.020833,-343.515495,-349.384544,-5.456566,-2.5881,3.000583,-0.043033


charge_1


,vacuum,supercell,N,1/N,E_def,E_bulk,mu_S_Mo-rich,VBM,E_form_uncorr,E_corr
0,vac_15,3x3x1,27,0.037037,-187.400635,-196.531426,-5.456566,-1.6657,2.008525,-0.235181
1,vac_20,3x3x1,27,0.037037,-186.104404,-196.528449,-5.456566,-2.5881,2.379379,-0.604896
2,vac_20,4x4x1,48,0.020833,-339.461205,-349.384544,-5.456566,-2.5881,1.878673,-0.158326


### 6.      CORRECTED defect formation energy:

Finally, we have all the pieces to evaluate the corrected defect formation energy!

* This final python script simply adds the correction terms to the uncorrected defect formation energies to give us the final results:

In [20]:
%cd /orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA

calc_Eform_corr.calc("/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA/",
                     "Eform_Svac.xlsx")
# python /path/to/calc_Eform_corr.py /path/to/Svac/GGA Eform_Svac.xlsx

/orange/hennig/annemarietan/precalculated_for_demo/MoS2/Svac/GGA


In [21]:
df = pandas.read_excel("Eform_Svac.xlsx", sheet_name=None)
for q in df.keys():
    print (q)
    display(df[q])

charge_0


,vacuum,supercell,N,1/N,E_def,E_bulk,mu_S_Mo-rich,VBM,E_form_corr
0,vac_15,3x3x1,27,0.037037,-189.728601,-196.531426,-5.456566,-1.6657,1.346259
1,vac_20,3x3x1,27,0.037037,-189.725299,-196.528449,-5.456566,-2.5881,1.346584
2,vac_20,4x4x1,48,0.020833,-342.600831,-349.384544,-5.456566,-2.5881,1.327147


charge_-1


,vacuum,supercell,N,1/N,E_def,E_bulk,mu_S_Mo-rich,VBM,E_form_uncorr,E_corr,E_form_corr
0,vac_15,3x3x1,27,0.037037,-189.757534,-196.531426,-5.456566,-1.6657,2.983025,0.015273,2.998299
1,vac_20,3x3x1,27,0.037037,-190.233532,-196.528449,-5.456566,-2.5881,3.426451,-0.413040,3.013411
2,vac_20,4x4x1,48,0.020833,-343.515495,-349.384544,-5.456566,-2.5881,3.000583,-0.043033,2.957550


charge_1


,vacuum,supercell,N,1/N,E_def,E_bulk,mu_S_Mo-rich,VBM,E_form_uncorr,E_corr,E_form_corr
0,vac_15,3x3x1,27,0.037037,-187.400635,-196.531426,-5.456566,-1.6657,2.008525,-0.235181,1.773344
1,vac_20,3x3x1,27,0.037037,-186.104404,-196.528449,-5.456566,-2.5881,2.379379,-0.604896,1.774483
2,vac_20,4x4x1,48,0.020833,-339.461205,-349.384544,-5.456566,-2.5881,1.878673,-0.158326,1.720347


You should see that the corrected formation energies for the charged defects are now also pretty well-converged. The vacuum dependence should essentially have been completely corrected, leaving only some in-plane supercell size dependence. In this case, the difference in formation energy evaluated with a 3x3 vs. 4x4 supercell should be around 50 meV, which is good enough for us at this point.